This program ateempts to predict the future price of a stock of apple

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"callmemaker","key":"acafd8d1d49f4c383b2c206fcd045b87"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d varpit94/apple-stock-data-updated-till-22jun2021


apple-stock-data-updated-till-22jun2021.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip apple-stock-data-updated-till-22jun2021.zip


Archive:  apple-stock-data-updated-till-22jun2021.zip
replace AAPL.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
df = pd.read_csv('AAPL.csv')

In [ ]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100323,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095089,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088110,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090291,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092908,73449600
...,...,...,...,...,...,...,...
10404,2022-03-18,160.509995,164.479996,159.759995,163.979996,163.979996,123351200
10405,2022-03-21,163.509995,166.350006,163.009995,165.380005,165.380005,95811400
10406,2022-03-22,165.509995,169.419998,164.910004,168.820007,168.820007,81532000
10407,2022-03-23,167.990005,172.639999,167.649994,170.210007,170.210007,98062700


In [ ]:
import plotly.express as px

# Asegúrate de que la columna 'Date' esté en formato datetime
df['Date'] = pd.to_datetime(df['Date'])

# Crea un gráfico interactivo con plotly
fig = px.line(df, x='Date', y='Close', title='Precio de Cierre de AAPL a lo largo del tiempo')

# Mejora el layout
fig.update_layout(
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    xaxis_rangeslider_visible=True  # Añade un slider para el rango de fechas
)

# Muestra el gráfico
fig.show()


The model

In [ ]:
model = RandomForestRegressor()

In [ ]:
#Training the model
X = df[['Open', 'High', 'Low', 'Volume']]
X = X[:int(len(df)-1)]
y = df['Close']
y = y[:int(len(df)-1)]
model.fit(X,y)

RandomForestRegressor()

In [ ]:
predictions = model.predict(X)
print(f'The model schore is: {model.score(X,y)}')

The model schore is: 0.9999833848443053


In [ ]:
import plotly.graph_objects as go

# Creando un rango de fechas para el eje x, excluyendo la última fecha ya que fue excluida del entrenamiento
dates = df['Date'][:int(len(df)-1)]

# Crear un gráfico de Plotly
fig = go.Figure()

# Agregar línea real de precios de cierre
fig.add_trace(go.Scatter(x=dates, y=y, mode='lines', name='Real Close Price'))

# Agregar línea de predicciones
fig.add_trace(go.Scatter(x=dates, y=predictions, mode='lines', name='Predicted Close Price'))

# Mejora del layout
fig.update_layout(title='Comparación del Precio de Cierre Real vs Predicción',
                  xaxis_title='Fecha',
                  yaxis_title='Precio de Cierre (USD)',
                  xaxis_rangeslider_visible=True)

# Mostrar el gráfico
fig.show()


Predictions

In [ ]:
new_data = df[['Open', 'High', 'Low', 'Volume']].tail(1)
prediction = model.predict(new_data)
print(f'The model predict the last row or day to be: {prediction}')

The model predict the last row or day to be: [172.51100116]


In [ ]:
print(f'Actual value is: {df[["Close"]].tail(1).values[0][0]}')


Actual value is: 174.070007


In [ ]:
import plotly.graph_objects as go

# Obteniendo el último valor real de cierre y la fecha correspondiente para etiquetar nuestro gráfico
actual_close_value = df["Close"].tail(1).values[0]
last_date = df["Date"].tail(1).values[0]

# Creando el gráfico de barras
fig = go.Figure()

# Agregando la barra de predicción
fig.add_trace(go.Bar(x=["Predicción"], y=[prediction[0]], name='Predicción', marker_color='indianred'))

# Agregando la barra del valor real
fig.add_trace(go.Bar(x=["Valor Real"], y=[actual_close_value], name='Valor Real', marker_color='lightsalmon'))

# Mejora del layout
fig.update_layout(title=f'Predicción vs Valor Real para {last_date}',
                  yaxis_title='Precio de Cierre (USD)',
                  xaxis_title='')

# Mostrar el gráfico
fig.show()
